In [1]:
# using DataFrames
# using GeoStats
# using GeoIO
# using Pkg
using NCDatasets, GLMakie
# using NetCDF
# using CairoMakie, ColorSchemes


In [2]:
# ================================================================
					# Plotting parameters
					ColourOption_No    = 1
					Linewidth          = 2
					height             = 400
					labelsize          = 20
					textcolor          = :blue
					textsize           = 20
					titlecolor         = :navyblue
					titlesize          = 18.0
					width              = height * 1.0
					xgridstyle         = :dash
					xgridvisible       = true
					xlabelSize         = 20
					xlabelpadding      = 5
					xminortickalign    = 1.0
					xminorticksvisible = true
					xtickalign         = 0.9 # 0 is inside and 1 is outside
					xticklabelrotation = π / 4.0
					xticksize          = 10
					xticksmirrored     = false
					xtickwidt          = 0.5
					xtrimspine         = false
					ygridstyle         = :dash
					ygridvisible       = false
					ylabelpadding      = xlabelpadding
					ylabelsize         = xlabelSize
					yminortickalign    = xminortickalign
					yminorticksvisible = true
					ytickalign         = xtickalign
					yticksize          = xticksize
					yticksmirrored     = false
					ytickwidt          = xtickwidt
					ytrimspine         = false

					Linewidth = 4
					xlabelSize = 30
					xticksize = 10
					xgridvisible = false
					Width = 800 # 800
					Height = 200


200

In [3]:
Path = raw"D:\JOE\MAIN\MODELS\WFLOW\Wflow.jl\data\output"
Path_Output = joinpath(Path, "output_moselle.nc")
Path_Scalar = joinpath(Path, "output_scalar_moselle.nc")
Path_Outstates = joinpath(Path, "outstates-moselle.nc")

Path_Input = joinpath(Path, "forcing-moselle.nc")

Path_Input = Path_Output


"D:\\JOE\\MAIN\\MODELS\\WFLOW\\Wflow.jl\\data\\output\\output_moselle.nc"

In [4]:
# NetCDF.ncinfo(Path_Static)
# All information of NCDataset
Output_NCDatasets = NCDatasets.NCDataset(Path_Input)


Dataset: D:\JOE\MAIN\MODELS\WFLOW\Wflow.jl\data\output\output_moselle.nc
Group: /

Dimensions
   time = 31
   lon = 291
   lat = 313
   layer = 4

Variables
  lon   (291)
    Datatype:    Float64 (Float64)
    Dimensions:  lon
    Attributes:
     long_name            = longitude
     standard_name        = longitude
     axis                 = X
     units                = degrees_east

  lat   (313)
    Datatype:    Float64 (Float64)
    Dimensions:  lat
    Attributes:
     long_name            = latitude
     standard_name        = latitude
     axis                 = Y
     units                = degrees_north

  layer   (4)
    Datatype:    Float64 (Float64)
    Dimensions:  layer
    Attributes:
     long_name            = layer_index
     standard_name        = layer_index
     axis                 = Z

  time   (31)
    Datatype:    Dates.DateTime (Float64)
    Dimensions:  time
    Attributes:
     units                = days since 1900-01-01 00:00:00
     calendar           

In [5]:
Keys = NCDatasets.keys(Output_NCDatasets)
Keys_Select = []

for iiKeys ∈ Keys
	Output = Output_NCDatasets[iiKeys]
	Dimensions = length(size(Output))
	println("$iiKeys   ; Dim=$Dimensions" )

	if Dimensions == 3
			push!(Keys_Select, String(iiKeys))
	end # Dimensions == 2
end # for iiKeys ∈ Keys

Keys_Select = String.(Keys_Select)
println()
@show Keys_Select


lon   ; Dim=1
lat   ; Dim=1
layer   ; Dim=1
time   ; Dim=1
ssf   ; Dim=3
interception   ; Dim=3
h_river   ; Dim=3
satwaterdepth   ; Dim=3
vwc   ; Dim=4
actevap   ; Dim=3
q_land   ; Dim=3
h_land   ; Dim=3
zi   ; Dim=3
ustorelayerdepth   ; Dim=4
tsoil   ; Dim=3
runoff   ; Dim=3
exfiltwater   ; Dim=3
q_river   ; Dim=3
canopystorage   ; Dim=3
transfer   ; Dim=3
q_river_av   ; Dim=3
to_river   ; Dim=3
snow   ; Dim=3
transpiration   ; Dim=3
snowwater   ; Dim=3
recharge   ; Dim=3
h_river_av   ; Dim=3

Keys_Select = ["ssf", "interception", "h_river", "satwaterdepth", "actevap", "q_land", "h_land", "zi", "tsoil", "runoff", "exfiltwater", "q_river", "canopystorage", "transfer", "q_river_av", "to_river", "snow", "transpiration", "snowwater", "recharge", "h_river_av"]


21-element Vector{String}:
 "ssf"
 "interception"
 "h_river"
 "satwaterdepth"
 "actevap"
 "q_land"
 "h_land"
 "zi"
 "tsoil"
 "runoff"
 ⋮
 "canopystorage"
 "transfer"
 "q_river_av"
 "to_river"
 "snow"
 "transpiration"
 "snowwater"
 "recharge"
 "h_river_av"

In [6]:
using GLMakie
GLMakie.activate!
Makie.inline!(false)  # Make sure to inline plots into Documenter output!


function HEATMAP_TIME(;Path=Path, NameOutput="q_land", Layer=1)
	Output_NCDatasets = NCDatasets.NCDataset(Path)

	Data = Output_NCDatasets[NameOutput]

	Data = Array(Data)

	Dimensions = length(size(Data))

	if Dimensions == 3
		N_Lon = size(Data)[1]
		N_Lat  = size(Data)[2]
		N_Time  = size(Data)[3]

	elseif Dimensions == 4
		N_Lon = size(Data)[1]
		N_Lat  = size(Data)[2]
		N_Time  = size(Data)[4]
	end

	Pmin, Pmax = extrema(x for x ∈ skipmissing(Data) if !isnan(x))
	@show Pmin Pmax

	function DATA_3D_2_2D(Data; iTime=iTime, Dimensions=Dimensions, Layer=Layer)
		if Dimensions == 4
			return Data[:,:, Layer, iTime]
		elseif Dimensions == 3
			return Data[:,:, iTime]
		end
	end


	Fig = Figure(size=(Width, Height * 4.0))

	Ax_1 = Axis(Fig[1, 1], title=NameOutput, xlabelsize=xlabelSize, ylabelsize=xlabelSize, xticksize=xticksize, xgridvisible=xgridvisible, ygridvisible=xgridvisible)

	sg = SliderGrid(Fig[2, 1],
	(label="iTime", range=1:1:N_Time, startvalue=1),
	width=550, tellheight=true)

	iTime = sg.sliders[1].value

	Data_Time = lift((iTime) -> DATA_3D_2_2D(Data; iTime=iTime, Dimensions), iTime)


	Data_Plot = heatmap!(Ax_1, 1:N_Lon, 1:N_Lat, Data_Time, colorrange=(Pmin, Pmax), colormap =:hawaii50)

	Colorbar(Fig[1, 2], Data_Plot; label=NameOutput, width=20, ticks = Pmin:(Pmax-Pmin)/5:Pmax)

	Fig
end



HEATMAP_TIME (generic function with 1 method)

In [7]:
# HEATMAP_TIME(;Path=Path_Input, NameOutput="h_av_land", Layer=1)
HEATMAP_TIME(;Path=Path_Input, NameOutput="ustorelayerdepth", Layer=1)
# HEATMAP_TIME(;Path=Path_Input, NameOutput="q_av_river", Layer=1)

# HEATMAP_TIME(;Path=Path_Output, NameOutput="ustorelayerdepth")


Pmin = 0.0f0
Pmax = 76.98983f0
